In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from MetabolomicsPaper.GeneralHelperFunctions import *
from ACSPaper.ACSPaperUtils import *
from sklearn.metrics import roc_curve, auc
import shap
from sklearn.model_selection import train_test_split

paper_dir = '/net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/'
curr_dir = os.path.join(paper_dir, 'fastqs_to_dropbox')
make_dir_if_not_exists(curr_dir)
os.chdir(paper_dir)

In [2]:
acs_mb_full_phyla = pd.read_csv(os.path.join('../TaxaComposition', 'acs_mb_full_phyla.csv'), index_col=0)
pnp_mb_full_phyla = pd.read_csv(os.path.join('../TaxaComposition', 'pnp_mb_full_phyla.csv'), index_col=0)

In [5]:
# total number of ACS
acs_full_data = pd.read_csv(os.path.join('data', 'ACS_full_data.csv'), index_col=0)
# total number of non-ACS
pnp1_full_data = pd.read_csv(os.path.join('data', 'pnp1_full_data.csv'), index_col=1)

Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.


In [8]:
pnp_mb = pnp1_full_data[(pnp1_full_data.Nextera=='True') & (pnp1_full_data.IsGenotek == 0)].dropna(subset=[pnp1_full_data.filter(regex='k__').columns[0]])
acs_mb = acs_full_data.dropna(subset=[acs_full_data.filter(regex='k__').columns[0]])

In [14]:
pnp_mb['FD'].sort_values()
acs_mb['FD'].sort_values()

RegistrationCode
3705508    FD2423
3401349    FD2424
3697558    FD2425
3485604    FD2427
3434612    FD2428
            ...  
3103104    FD3820
3139022    FD3821
3889402    FD3822
3025867    FD3823
3713651    FD3824
Name: FD, Length: 199, dtype: object

In [27]:
novaseq = '/net/mraid08/export/genie/LabData/Data/MBPipeline/NovaSeq/tmp2/UNT/' 
hanson = '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/'
pnp_files = []
acs_files = []

d = novaseq
files = [f for f in os.listdir(d) if 'FD' in f]
# pnp_files += [os.path.join(d, f) for f in files if 'FD' + f.split('FD')[1].split('_')[0] in pnp_mb['FD']]
acs_files += [os.path.join(d, f) for f in files if ('FD' + f.split('FD')[1].split('.fastq')[0].split('_')[0]) in acs_mb['FD'].values]

d = hanson
files = [f for f in os.listdir(d) if 'FD' in f]
pnp_files += [os.path.join(d, f) for f in files if ('FD' + f.split('FD')[1].split('.fastq')[0]) in pnp_mb['FD'].values]
acs_files += [os.path.join(d, f) for f in files if ('FD' + f.split('FD')[1].split('.fastq')[0]) in acs_mb['FD'].values]
# os.listdir()
# 

In [39]:
pnp_mb['ACS sample'] = False
acs_mb['ACS sample'] = True
pd.concat((pnp_mb[['FD', 'ACS sample']], acs_mb[['FD', 'ACS sample']]), axis=0).to_csv(os.path.join(paper_dir, 'data', 'RegID-FD.csv'))



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
import shutil
for f in tqdm_notebook(pnp_files+acs_files):
    shutil.copyfile(f, os.path.join(curr_dir, os.path.basename(f)))

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [40]:
import shutil
for f in tqdm_notebook(acs_files):
    shutil.copyfile(f, os.path.join(curr_dir, os.path.basename(f)))

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [32]:
os.path.basename(pnp_files[0])

'210867_FD1694.fastq.gz'

In [30]:
pnp_files

['/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/210867_FD1694.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/921543_FD2233.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/774511_FD2235.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/507873_FD2181.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/128161_FD2160.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/876581_FD1766.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/994875_FD2225.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/913400_FD2128.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/113719_FD1689.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/44114_FD2112.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/392420_FD2120.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/853351_FD2053.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/740046_FD1702.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/120636_FD2083.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/696383_FD2020.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/720396_FD1980.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/403931_FD2016.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/200924_FD2270.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/121123_FD2022.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/465440_FD1995.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/315302_FD1690.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/465856_FD1707.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/338887_FD1698.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/592536_FD1973.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/613001_FD2212.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/298961_FD2099.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/515714_FD2164.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/987948_FD2117.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/896645_FD2201.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/928726_FD1977.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/738799_FD1992.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/27149_FD2152.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/568087_FD2079.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/738097_FD2229.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/312685_FD1674.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/980296_FD2211.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/13060_FD2276.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/148494_FD1770.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/968418_FD2195.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/876636_FD2239.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/628035_FD2037.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/927003_FD2191.fastq.gz',
 '/net/mraid08/export/genie/LabData/Data/MBPipeline/Hanson/tmp2/UNT/973938_FD2244.fastq.gz',
 '/n